In [53]:
import pandas as pd
df = pd.read_csv("Morakot_data/wind_200907.csv", header=None)

    # 第 0 列為小時 → 取 1~24
hours = df.iloc[0, 1:25].astype(int).tolist()  # 1~24
    

In [81]:
#for rain data

import pandas as pd

def convert_cwa_hourly_table(csv_path, year, month):
    df = pd.read_csv(csv_path, header=None)

    # 第 0 列為小時 → 取 1~24
    hours = df.iloc[0, 1:25].astype(int).tolist()  # 1~24
    
    # 真正資料從第 1 列開始（每天）
    df_days = df.iloc[1:31, 0:25].copy()

    df_days.columns = ["day"] + hours

    # 移除 "總和" 欄（如果存在）
    if df_days.columns[-1] == 24:
        pass  # 正常無事
    else:
        df_days = df_days.iloc[:, :-1]

    # 長格式
    df_long = df_days.melt(id_vars="day", var_name="hour", value_name="rain")

    # day, hour → datetime
    df_long["day"] = df_long["day"].astype(int)
    df_long["hour"] = df_long["hour"].astype(int)

    df_long["datetime"] = pd.to_datetime(
        {
            "year": year,
            "month": month,
            "day": df_long["day"],
            "hour": df_long["hour"] - 1,  # CWA hour=1 → 00:00
        }
    )

    df_long = df_long[["datetime", "rain"]].sort_values("datetime")
  
    return df_long

# 使用你的檔案
df = convert_cwa_hourly_table("Morakot_data/rain_200909.csv", 2009, 9)
  # 修正 df_weather["rain"] 中的非數值（例如 "T"）
df["rain"] = (
    df["rain"]
    .replace("T", "0.0")     # CWA 的 trace amount
    .replace("-", "0.0")     # 偶爾會出現 "-"; 若沒有也不會壞
)

# 轉 float
df["rain"] = df["rain"].astype(float)
df.to_csv("Morakot_data/rain_200909_pd.csv", index=False)
df.head(10)


,datetime,rain
0,2009-09-01 00:00:00,0.0
30,2009-09-01 01:00:00,0.0
60,2009-09-01 02:00:00,0.0
90,2009-09-01 03:00:00,0.0
120,2009-09-01 04:00:00,0.0
150,2009-09-01 05:00:00,0.0
180,2009-09-01 06:00:00,0.0
210,2009-09-01 07:00:00,0.0
240,2009-09-01 08:00:00,0.0
270,2009-09-01 09:00:00,0.0


In [ ]:
#for solar data

import pandas as pd

def convert_cwa_hourly_table(csv_path, year, month):
    df = pd.read_csv(csv_path, header=None)

    # 第 0 列為小時 → 取 1~24
    hours = df.iloc[0, 1:25].astype(int).tolist()  # 1~24
    
    # 真正資料從第 1 列開始（每天）
    df_days = df.iloc[1:32, 0:25].copy()

    df_days.columns = ["day"] + hours

    # 移除 "總和" 欄（如果存在）
    if df_days.columns[-1] == 24:
        pass  # 正常無事
    else:
        df_days = df_days.iloc[:, :-1]

    # 長格式
    df_long = df_days.melt(id_vars="day", var_name="hour", value_name="solar")

    # day, hour → datetime
    df_long["day"] = df_long["day"].astype(int)
    df_long["hour"] = df_long["hour"].astype(int)

    df_long["datetime"] = pd.to_datetime(
        {
            "year": year,
            "month": month,
            "day": df_long["day"],
            "hour": df_long["hour"] - 1,  # CWA hour=1 → 00:00
        }
    )

    df_long = df_long[["datetime", "solar"]].sort_values("datetime")

    return df_long

# 使用你的檔案
df = convert_cwa_hourly_table("Morakot_data/solar_200908.csv", 2009, 8)
df.to_csv("Morakot_data/solar_200908_pd.csv", index=False)
df.head(10)


,datetime,solar
0,2009-08-01 00:00:00,0.00
31,2009-08-01 01:00:00,0.00
62,2009-08-01 02:00:00,0.00
93,2009-08-01 03:00:00,0.00
124,2009-08-01 04:00:00,0.00
155,2009-08-01 05:00:00,0.00
186,2009-08-01 06:00:00,0.17
217,2009-08-01 07:00:00,0.72
248,2009-08-01 08:00:00,0.81
279,2009-08-01 09:00:00,0.84


In [71]:
#for wind data
import pandas as pd
import numpy as np

def convert_cwa_wind_table(csv_path, year, month):
    df = pd.read_csv(csv_path, header=None)

    # 第 0 列: 小時 "1"~"24"
    hour_cols = df.iloc[0, 1:25].astype(int).tolist()

    # 從第 1 列開始為每天資料
    df_days = df.iloc[1:31, 0:25].copy()
    df_days.columns = ["day"] + hour_cols

    # -------- 缺值處理 function --------
    def parse_ws_wd(cell):
        if not isinstance(cell, str):
            return np.nan, np.nan
        
        cell = cell.strip()

        # 如果含有 X → 缺值
        if "X" in cell or cell == "":
            return np.nan, np.nan
        
        if "/" not in cell:
            return np.nan, np.nan

        try:
            ws, wd = cell.split("/")
            ws = ws.strip()
            wd = wd.strip()
            return float(ws), float(wd)
        except:
            return np.nan, np.nan

    # 展開成 long format
    df_long = df_days.melt(id_vars="day", var_name="hour", value_name="ws_wd")

    # 拆出 wind_speed & wind_dir
    df_long[["wind_speed", "wind_dir"]] = df_long["ws_wd"].apply(
        lambda x: pd.Series(parse_ws_wd(x))
    )

    # CWA hour=1 表示 00:00 → 所以改成 hour-1
    df_long["hour"] = df_long["hour"].astype(int) - 1
    df_long["day"] = df_long["day"].astype(int)

    # 建立 datetime
    df_long["datetime"] = pd.to_datetime(
        {
            "year": year,
            "month": month,
            "day": df_long["day"],
            "hour": df_long["hour"],
        },
        errors="coerce"
    )

    # 排序 & 保留必要欄位
    df_long = df_long[["datetime", "wind_speed", "wind_dir"]].sort_values("datetime")

    return df_long

# === 使用 ===
df_wind = convert_cwa_wind_table("Morakot_data/wind_200909.csv", 2009, 9)
df_wind.to_csv("Morakot_data/wind_200909_pd.csv", index=False)
print(df_wind.head(12))


               datetime  wind_speed  wind_dir
0   2009-09-01 00:00:00         1.8     130.0
30  2009-09-01 01:00:00         0.8     110.0
60  2009-09-01 02:00:00         0.6     100.0
90  2009-09-01 03:00:00         1.4     120.0
120 2009-09-01 04:00:00         0.3     110.0
150 2009-09-01 05:00:00         0.2       0.0
180 2009-09-01 06:00:00         0.1       0.0
210 2009-09-01 07:00:00         0.8      90.0
240 2009-09-01 08:00:00         1.6     310.0
270 2009-09-01 09:00:00         2.0     350.0
300 2009-09-01 10:00:00         1.9     250.0
330 2009-09-01 11:00:00         3.0     250.0


In [73]:
#for filling missing wind speed data
csv_path = "Morakot_data/wind_200909_pd.csv"
df = pd.read_csv(csv_path, header=None)
df_wind["wind_speed"] = (
    df_wind["wind_speed"]
    .interpolate()
    .fillna(method="bfill")
    .fillna(method="ffill")
)
df_wind.to_csv("Morakot_data/wind_200909_pd.csv", index=False)

C:\Users\Yifang\AppData\Local\Temp\ipykernel_21036\3185577205.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="bfill")
C:\Users\Yifang\AppData\Local\Temp\ipykernel_21036\3185577205.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")


In [57]:
import pandas as pd
df_1 = pd.read_csv("Morakot_data/wind_200907_pd.csv")
df_2= pd.read_csv("Morakot_data/rain_200907_pd.csv")
df_3 = pd.read_csv("Morakot_data/solar_200907_pd.csv")
#df_wind_200908 = df_wind_200908.reset_index()
#df_wind_200909 = df_wind_200909.reset_index()
print(df_1.shape)
print(df_2.shape)
print(df_3.shape)


(744, 3)
(744, 2)
(744, 2)


In [82]:
import pandas as pd
import numpy as np
from microgrid.models import TimeSeriesInput, HazardProfile


# =======================================================
# 1. 讀取並合併 7~9 月的 9 個 CSV 變成一個 DataFrame
# =======================================================

def load_three_months_weather(path_wind, path_rain, path_solar):
    df_w = pd.read_csv(path_wind, parse_dates=["datetime"])
    df_r = pd.read_csv(path_rain, parse_dates=["datetime"])
    df_s = pd.read_csv(path_solar, parse_dates=["datetime"])

    # 合併：以 time 為 key
    df = df_w.merge(df_r, on="datetime", how="inner") \
             .merge(df_s, on="datetime", how="inner")

    # 排序
    df = df.sort_values("datetime").reset_index(drop=True)

    return df


# =======================================================
# 2. 將 7~9 月資料讀進來
# =======================================================

# 你的檔案名稱（修改成自己的）
df_jul = load_three_months_weather(
    "Morakot_data/wind_200907_pd.csv",
    "Morakot_data/rain_200907_pd.csv",
    "Morakot_data/solar_200907_pd.csv"
)
df_aug = load_three_months_weather(
    "Morakot_data/wind_200908_pd.csv",
    "Morakot_data/rain_200908_pd.csv",
    "Morakot_data/solar_200908_pd.csv"
)
df_sep = load_three_months_weather(
    "Morakot_data/wind_200909_pd.csv",
    "Morakot_data/rain_200909_pd.csv",
    "Morakot_data/solar_200909_pd.csv"
)

# 合併成完整的 2009/07–09 資料
df_weather = pd.concat([df_jul, df_aug, df_sep], axis=0)
df_weather = df_weather.sort_values("datetime").reset_index(drop=True)

print("Loaded Morakot weather data:", df_weather.shape)

print(df_weather.shape)

# =======================================================
# 3. 轉換為容量因子 CF
# =======================================================

def compute_cf_WT(v):
    """Compute wind turbine capacity factor from wind speed."""
    cut_in = 3
    rated = 12
    cut_out = 25

    if v < cut_in:
        return 0.0
    if v >= cut_out:
        return 0.0
    if v < rated:
        wind_cf = (v - cut_in) / (rated - cut_in)
        wind_cf = round(wind_cf, 2)
        return wind_cf
    return 1.0

# 風力 CF

df_weather["cf_WT"] = df_weather["wind_speed"].apply(compute_cf_WT)

# PV CF：
# solar 單位：MJ/m² per hour
# 轉成 W/m²
df_weather["irr_Wm2"] = (df_weather["solar"] * 277.7778).round(2)

# 轉成容量因子 CF
df_weather["cf_PV"] = (df_weather["irr_Wm2"] / 1000.0).round(2)

# 確保 CF 在 0~1 之間
df_weather["cf_PV"] = df_weather["cf_PV"].clip(lower=0, upper=1)


# =======================================================
# 4. 產生 hour index（EMS 用）
# =======================================================

df_weather["hour"] = df_weather["datetime"].dt.hour


# =======================================================
# 5. 建立 TimeSeriesInput 物件，替換模型輸入
# =======================================================

def build_time_input(df_weather, demand_series):
    """
    df_weather: 需包含 wind, rain, cf_WT, cf_PV
    demand_series: 小時需求（長度需一致）
    """

    N = len(df_weather)
    if len(demand_series) != N:
        raise ValueError(f"demand_series length {len(demand_series)} != df_weather length {N}")
  

    # 若沒有 time_h（小時），自動產生 0~23 循環
    if "hour" in df_weather.columns:
        time_h = df_weather["hour"].values.tolist()
    else:
        time_h = [(i % 24) for i in range(N)]

    # 若沒有 solar_irradiance，就用 cf_PV * 1000 反推（模型不會真的用它，只是 HazardProfile 需要）
    if "irr_Wm2" in df_weather.columns:
        solar_raw = df_weather["irr_Wm2"].values.tolist()
    else:
        solar_raw = (df_weather["cf_PV"] * 1000).values.tolist()

    ts = TimeSeriesInput(
        cf_WT=df_weather["cf_WT"].values.tolist(),
        cf_PV=df_weather["cf_PV"].values.tolist(),
        demand=demand_series,
        hours=time_h,
    )

    return ts

from microgrid.models import HazardProfile

def build_hazard_from_weather(df_weather: pd.DataFrame) -> HazardProfile:
    """
    從 df_weather 生成 HazardProfile，給 DisturbanceScenario 使用。
    
    需求欄位：
      wind  : 風速 (m/s)
      rain  : 降雨量 (mm/hr)
      solar (或 solar_raw 或 solar_Wm2)：日射量，用來當作 solar_irradiance
    """
    N = len(df_weather)

    # 時間軸：用「模擬小時索引」0,1,2,...,N-1
    time_h = list(range(N))

    # 風速
    if "wind" not in df_weather.columns:
        raise KeyError("df_weather 缺少 'wind' 欄位（m/s）")
    wind_speed = df_weather["wind"].astype(float).tolist()

    # 降雨
    if "rain" not in df_weather.columns:
        raise KeyError("df_weather 缺少 'rain' 欄位（mm/hr）")
    rainfall = df_weather["rain"].astype(float).tolist()

    # 日射量：優先用 solar_Wm2，其次 solar_raw，其次 solar
    if "solar_Wm2" in df_weather.columns:
        solar_irr = df_weather["solar_Wm2"].astype(float).tolist()
    elif "solar_raw" in df_weather.columns:
        solar_irr = df_weather["solar_raw"].astype(float).tolist()
    elif "solar" in df_weather.columns:
        solar_irr = df_weather["solar"].astype(float).tolist()
    else:
        # 沒有的話就先給 0，至少不會壞掉
        solar_irr = [0.0] * N

    hazard = HazardProfile(
        time_h=time_h,
        wind_speed=wind_speed,
        rainfall=rainfall,
        solar_irradiance=solar_irr,
    )
    return hazard


# =======================================================
# 6. 替換你模型使用的 time_input
# =======================================================

import pandas as pd

# 讀取你的 24 小時需求曲線
df_demand = pd.read_csv("Demand_profile_1D.csv")

# 假設欄位名稱叫 "demand_kW" 或只有一欄（你可視狀況調整）
if "D_kW" in df_demand.columns:
    demand_24h = df_demand["D_kW"].tolist()
else:
    # 若只有一欄，取出第一欄
    demand_24h = df_demand.iloc[:, 0].tolist()

# 確保是 24 筆資料
if len(demand_24h) != 24:
    raise ValueError(f"Demand_profile_1D.csv 不是 24 小時資料！共 {len(demand_24h)} 筆")

# 依照莫拉克資料長度展開 demand
N = len(df_weather)  # 例如 2208 小時（92 天）
demand_series = (demand_24h * (N // 24 + 1))[:N]

# 建立 time_input
time_input = build_time_input(df_weather, demand_series)

print("=== New time_input loaded with Morakot 2009 data ===")
print("Hours:", len(time_input.cf_PV))




Loaded Morakot weather data: (2208, 5)
(2208, 5)
=== New time_input loaded with Morakot 2009 data ===
Hours: 2208


In [83]:
import pandas as pd

# df_weather 已經建好（含 datetime, wind, rain, solar, cf_WT, cf_PV）
# demand_series 是 list / array，長度必須 = len(df_weather)

# 加入 demand 欄位
df_weather["demand"] = demand_series

# 輸出 CSV
output_path = "Morakot_weather_with_demand.csv"
df_weather.to_csv(output_path, index=False, encoding="utf-8-sig")

print("已成功輸出：", output_path)
print(df_weather.head())


已成功輸出： Morakot_weather_with_demand.csv
             datetime  wind_speed  wind_dir  rain  solar  cf_WT  irr_Wm2  \
0 2009-07-01 00:00:00         2.0     190.0   0.0    0.0    0.0      0.0   
1 2009-07-01 01:00:00         2.5     190.0   0.0    0.0    0.0      0.0   
2 2009-07-01 02:00:00         2.3     170.0   0.0    0.0    0.0      0.0   
3 2009-07-01 03:00:00         1.8     150.0   0.0    0.0    0.0      0.0   
4 2009-07-01 04:00:00         1.4     140.0   0.0    0.0    0.0      0.0   

   cf_PV  hour  demand  
0    0.0     0   48000  
1    0.0     1   47500  
2    0.0     2   47000  
3    0.0     3   46500  
4    0.0     4   46500  
